In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

In [2]:
# Load .env environment variables
load_dotenv()

True

In [3]:
# Set alpaca api key and secret 
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)


In [12]:
# Set the tickers
microsoft = ["MSFT"]

In [5]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [6]:
# Format current date as ISO format
today = pd.Timestamp("2022-06-01", tz="America/New_York").isoformat()

In [13]:
# Get current closing prices for FB and TWTR
df_microsoft = alpaca.get_bars(
    microsoft,
    timeframe,
    start = today,
    end = today
).df

# Display sample data
df_microsoft

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2022-06-01 04:00:00+00:00,275,277.69,270.04,272.42,25108530,318649,273.328471,MSFT


In [8]:
# Format start and end dates as ISO format for five year period
start = pd.Timestamp("2017-06-01", tz="America/New_York").isoformat()
end = pd.Timestamp("2022-06-01", tz="America/New_York").isoformat()

In [14]:
# Get closing prices for MSFT and TSLA from the five years
df_microsoft_daily = alpaca.get_bars(
    microsoft,
    timeframe,
    start = start,
    end = end,
    limit = 10000
).df

# Display sample data
df_microsoft_daily.head(10)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-06-01 04:00:00+00:00,70.24,70.61,69.451,70.10,21922908,115317,69.906282,MSFT
2017-06-02 04:00:00+00:00,70.44,71.86,70.240,71.76,35470261,152145,71.359913,MSFT
2017-06-05 04:00:00+00:00,71.97,72.89,71.810,72.28,33317996,137660,72.420495,MSFT
2017-06-06 04:00:00+00:00,72.30,72.62,72.270,72.52,31511065,140546,72.465401,MSFT
2017-06-07 04:00:00+00:00,72.67,72.77,71.950,72.39,22302769,122842,72.316942,MSFT
2017-06-08 04:00:00+00:00,72.42,72.52,71.500,71.95,26179442,136478,71.981665,MSFT
2017-06-09 04:00:00+00:00,72.02,72.08,68.590,70.32,49188196,287856,70.282896,MSFT
2017-06-12 04:00:00+00:00,69.29,69.94,68.130,69.78,47762073,287698,69.223695,MSFT
2017-06-13 04:00:00+00:00,70.04,70.82,69.960,70.65,25258687,146756,70.528400,MSFT


In [15]:
# Reorganize the DataFrame
# Separate ticker data
#MSFT = df_portfolio_years[df_portfolio_years['symbol']=='MSFT'].drop('symbol', axis=1)
#TSLA = df_portfolio_years[df_portfolio_years['symbol']=='TSLA'].drop('symbol', axis=1)
# Concatenate the ticker DataFrames
#df_portfolio_years = pd.concat([MSFT, TSLA],axis=1, keys=['MSFT','TSLA'])
#df_portfolio_years